In [8]:
%pip install --upgrade setuptools wheel 

  Using cached wheel-0.44.0-py3-none-any.whl.metadata (2.3 kB)
Using cached wheel-0.44.0-py3-none-any.whl (67 kB)
Note: you may need to restart the kernel to use updated packages.


In [8]:
%pip install wheel git+https://github.com/heebp/pykrx.git pymysql mysql-connector-python sqlalchemy

Note: you may need to restart the kernel to use updated packages.
  Cloning https://github.com/heebp/pykrx.git to c:\users\student\appdata\local\temp\pip-req-build-m2tsaz35
  Resolved https://github.com/heebp/pykrx.git to commit 5ef35bfd2095c874b498ca45e9d8cefef6e43afb
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
   ---------------------------------------- 0.0/14.3 MB ? eta -:--:--
   ---------------------------------------  14.2/14.3 MB 73.9 MB/s eta 0:00:01
   ---------------------------------------- 14.3/14.3 MB 59.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 2.1/2.1 MB 57.0 MB/s eta 0:00:00
Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)


  Running command git clone --filter=blob:none --quiet https://github.com/heebp/pykrx.git 'C:\Users\student\AppData\Local\Temp\pip-req-build-m2tsaz35'


In [12]:
from pykrx import stock
from pykrx import bond
import os
from dotenv import load_dotenv
import pandas as pd

In [17]:
load_dotenv()

DB_HOST = os.getenv("DB_HOST")
DB_USERNAME = os.getenv("DB_USERNAME")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_SCHEME = os.getenv("DB_SCHEME")

In [30]:
df1 = stock.get_market_ohlcv("20240926", market="KOSPI")
df1

,시가,고가,저가,종가,거래량,거래대금,등락률
티커,,,,,,,
095570,4790,4790,4730,4745,56083,266282560,-0.21
006840,12830,13020,12830,12990,1718,22291400,1.33
027410,3535,3555,3535,3550,16761,59448650,0.42
282330,120000,120300,119000,120200,59894,7167648150,0.33
138930,9080,9290,9080,9290,1558581,14294479590,2.77
...,...,...,...,...,...,...,...
079980,3000,3045,2985,3025,20352,61225865,1.34
005010,4150,4235,4150,4220,198156,830621870,1.93
000540,3680,3750,3675,3705,39818,147699230,0.68


In [31]:
df2 = stock.get_market_ohlcv("20240926", market="KOSDAQ")
df2

,시가,고가,저가,종가,거래량,거래대금,등락률
티커,,,,,,,
060310,2140,2195,2140,2190,413297,897370390,2.34
054620,5450,5680,5400,5650,20739,114720770,4.05
265520,19280,20800,19180,20550,594213,11991257150,8.73
211270,11030,11140,10890,11100,44154,488313550,2.30
109960,533,539,523,531,300790,158988075,1.34
...,...,...,...,...,...,...,...
024060,17560,18340,17260,17490,4248481,75501127280,-7.36
010240,4865,4955,4850,4880,6718,32907235,0.31
189980,1800,1831,1793,1826,88569,160854427,1.44


In [61]:
from sqlalchemy import create_engine, MetaData, Table
# MySQL 데이터베이스 연결 설정
db_username = DB_USERNAME
db_password = DB_PASSWORD
db_host = DB_HOST
db_name = DB_SCHEME

# 데이터프레임 컬럼명과 데이터베이스 컬럼명을 매핑하는 딕셔너리 생성

# SQLAlchemy 엔진 생성
engine = create_engine(f'mysql+pymysql://{db_username}:{db_password}@{db_host}/{db_name}')

# 메타데이터 객체 생성
metadata = MetaData()

table_name = 'STOCK_HISTORY'
table = Table(table_name, metadata, autoload_with=engine)

# 테이블 컬럼명 확인
db_columns = [column.name for column in table.columns]

db_columns


['STOCK_HISTORY_ID',
 'STOCK_ID',
 'SHORT_CODE',
 'CURRENT_PRICE',
 'CREATED_AT',
 'UPDATED_AT']

In [64]:
from sqlalchemy import create_engine, MetaData, Table
from sqlalchemy.orm import sessionmaker

# MySQL 데이터베이스 연결 설정
engine = create_engine(f'mysql+pymysql://{db_username}:{db_password}@{db_host}/{db_name}')
# 세션 생성
Session = sessionmaker(bind=engine)
session = Session()

# 메타데이터 및 stock 테이블 정의
metadata = MetaData()
stock_table = Table('STOCK', metadata, autoload_with=engine)

# stock 테이블에서 short_code와 stock_id 조회
query = session.query(stock_table.c.SHORT_CODE, stock_table.c.STOCK_ID).all()

# short_code와 stock_id를 매핑한 딕셔너리 생성
short_code_to_stock_id_map = {row.SHORT_CODE: row.STOCK_ID for row in query}

# 결과 출력
print(short_code_to_stock_id_map)

# 세션 종료
session.close()

{'000140': 3667, '000145': 3668, '000150': 3669, '000155': 3670, '000157': 3671, '000180': 3672, '000210': 3673, '000215': 3674, '000220': 3675, '000225': 3676, '000227': 3677, '000230': 3678, '000240': 3679, '000270': 3680, '000300': 3681, '000320': 3682, '000325': 3683, '000370': 3684, '000390': 3685, '000400': 3686, '000430': 3687, '000480': 3688, '000490': 3689, '000500': 3690, '000520': 3691, '000540': 3692, '000545': 3693, '000590': 3694, '000640': 3695, '000650': 3696, '000660': 3697, '000670': 3698, '000680': 3699, '000700': 3700, '000720': 3701, '000725': 3702, '000760': 3703, '000810': 3704, '000815': 3705, '000850': 3706, '000860': 3707, '000880': 3708, '000885': 3709, '00088K': 3710, '000890': 3711, '000910': 3712, '000950': 3713, '000970': 3714, '000990': 3715, '001020': 3716, '001040': 3717, '001045': 3718, '00104K': 3719, '001060': 3720, '001065': 3721, '001067': 3722, '001070': 3723, '001080': 3724, '001120': 3725, '001130': 3726, '001140': 3727, '001200': 3728, '001210

In [70]:
result = pd.concat([df1, df2], axis=0)

result = result.reset_index()


column_mapping = {
    '티커': 'SHORT_CODE',
    '시가': 'OPEN_PRICE',
    '종가': 'CURRENT_PRICE',
    '고가': 'HIGH_PRICE',
    '저가': 'LOW_PRICE',
}

# 컬럼명 매핑 적용
result.rename(columns=column_mapping, inplace=True)

result['STOCK_ID'] = result['SHORT_CODE'].map(short_code_to_stock_id_map)

result['STOCK_ID'] = result['STOCK_ID'].astype(int)

result

KeyError: 'stock_id'

In [21]:
import pymysql

# MySQL 연결 설정
connection = pymysql.connect(
    host=DB_HOST,     # MySQL 서버 주소
    user=DB_USERNAME,       # MySQL 사용자 이름
    password=DB_PASSWORD,   # MySQL 비밀번호
    database=DB_SCHEME    # 사용할 데이터베이스 이름
)

In [22]:
# SQL 쿼리 실행
cursor = connection.cursor()
cursor.execute("SELECT * FROM SUB_CATEGORY")
result = cursor.fetchall()

for row in result:
    print(row)

(1, 1, '4차 산업', datetime.datetime(2024, 9, 26, 4, 50, 10), None)
(2, 1, 'IT기기', datetime.datetime(2024, 9, 26, 4, 50, 10), None)
(3, 1, '반도체', datetime.datetime(2024, 9, 26, 4, 50, 10), None)
(4, 1, '전지/배터리', datetime.datetime(2024, 9, 26, 4, 50, 10), None)
(5, 1, '통신/네트워크', datetime.datetime(2024, 9, 26, 4, 50, 10), None)
(6, 1, '플랫폼', datetime.datetime(2024, 9, 26, 4, 50, 10), None)
(7, 2, '석유/화학', datetime.datetime(2024, 9, 26, 4, 50, 10), None)
(8, 2, '신재생에너지', datetime.datetime(2024, 9, 26, 4, 50, 10), None)
(9, 2, '에너지', datetime.datetime(2024, 9, 26, 4, 50, 10), None)
(10, 3, 'ETF', datetime.datetime(2024, 9, 26, 4, 50, 11), None)
(11, 3, '채권', datetime.datetime(2024, 9, 26, 4, 50, 11), None)
(12, 3, '선물옵션', datetime.datetime(2024, 9, 26, 4, 50, 11), None)
(13, 3, '벤처투자/IPO', datetime.datetime(2024, 9, 26, 4, 50, 11), None)
(14, 3, '금융', datetime.datetime(2024, 9, 26, 4, 50, 11), None)
(15, 3, '투자전략', datetime.datetime(2024, 9, 26, 4, 50, 11), None)
(16, 4, '부동산/리츠', datetime.da

In [ ]:
cursor.close()
connection.close()